In [1]:
import sys
!{sys.executable} -m pip install -U turicreate matplotlib opencv-python
import turicreate as tc
import numpy as np
import os
import glob

     |████████████████████████████████| 7.2 MB 3.4 MB/s eta 0:00:01     |▌                               | 112 kB 3.4 MB/s eta 0:00:03     |█████████▉                      | 2.2 MB 3.4 MB/s eta 0:00:02


  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.4
    Uninstalling matplotlib-3.3.4:
      Successfully uninstalled matplotlib-3.3.4


In [2]:
from PIL import Image
from matplotlib import pyplot as plt
import cv2
classes = ["arrow", "circle", "cycle"]
for ml_class in classes:
    images = []
    image_labels = []
    files = glob.glob (f"data/images/{ml_class}/*")
    for image_file in files:
        im = Image.open(image_file)
        im = np.array(im)
        images.append(im)

    images = np.array(images)
    np.save(f'data/bitmaps/{ml_class}.npy', images)

In [3]:
# mostly taken from: https://github.com/apple/turicreate/blob/master/userguide/drawing_classifier/data-preparation.md

random_state = np.random.RandomState(100)

# Change if applicable
data_dir = 'data'
bitmaps_dir = os.path.join(data_dir, 'bitmaps')
sframes_dir = os.path.join(data_dir, 'sframes')
npy_ext = '.npy'
num_examples_per_class = 200
num_classes = len(classes)

def build_bitmap_sframe():
    bitmaps_list, labels_list = [], []
    for class_name in classes:
        class_data = np.load(os.path.join(bitmaps_dir, class_name + npy_ext))
        random_state.shuffle(class_data)
        class_data_selected = class_data[:num_examples_per_class]
        for np_pixel_data in class_data_selected:
            FORMAT_RAW = 2
            bitmap = tc.Image(_image_data = np_pixel_data.tobytes(),
                              _width = np_pixel_data.shape[1],
                              _height = np_pixel_data.shape[0],
                              _channels = 1,
                              _format_enum = FORMAT_RAW,
                              _image_data_size = np_pixel_data.size)
            bitmaps_list.append(bitmap)
            labels_list.append(class_name)

    sf = tc.SFrame({"drawing": bitmaps_list, "label": labels_list})
    sf.save(os.path.join(sframes_dir, "bitmap_data.sframe"))
    return sf 

sf = build_bitmap_sframe()

In [4]:
sf.explore()

/Users/marekfort/opt/miniconda3/envs/automata-editor/lib/python3.8/site-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,drawing,label
0,,arrow
1,,arrow
2,,arrow
3,,arrow
4,,arrow
5,,arrow
6,,arrow
7,,arrow
8,,arrow
9,,arrow


In [5]:
SFRAME_PATH = "data/sframes/bitmap_data.sframe"
data =  tc.SFrame(SFRAME_PATH)

In [6]:
data.explore()

/Users/marekfort/opt/miniconda3/envs/automata-editor/lib/python3.8/site-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,drawing,label
0,,arrow
1,,arrow
2,,arrow
3,,arrow
4,,arrow
5,,arrow
6,,arrow
7,,arrow
8,,arrow
9,,arrow


In [7]:
train_data, test_data = data.random_split(0.7)

In [8]:
model = tc.drawing_classifier.create(train_data, 'label', max_iterations=20)

Download completed: /var/folders/yc/57fqwfv971x9d3wwlvgllqsh0000gn/T/model_cache/drawing_classifier_pre_trained_model_245_classes_v0.mlmodel


Automatically generating validation set from 5% of the data.

Using CPU to create model.

+--------------+----------------+---------------------+--------------+

| Iteration    | Train Accuracy | Validation Accuracy | Elapsed Time |

...

| 5            | 0.770408       | 0.909091            | 1.48s        |

| 6            | 0.816327       | 0.954545            | 1.70s        |

| 11           | 0.969388       | 1                   | 3.08s        |

| 20           | 0.992347       | 1                   | 5.34s        |

+--------------+----------------+---------------------+--------------+

In [101]:
model

Class                                    : DrawingClassifier

Schema
------
Number of classes                        : 3
Feature column                           : drawing
Target column                            : label

Training summary
----------------
Training Iterations                      : 20
Training Accuracy                        : 1.0
Validation Accuracy                      : 0.8824
Training Time                            : 13.7598
Number of Examples                       : 153

In [9]:
predictions = model.predict(test_data)

In [10]:
metrics = model.evaluate(test_data)
metrics.explore()

Resizing image data... Done.


In [11]:
model.save("automata-editor.model")

In [12]:
model.export_coreml("AutomataClassifier.mlmodel")